In [1]:
import json
import os
import requests

import dash_bootstrap_components as dbc
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

from dash import html
from dash import dcc
from pathlib import Path
from shapely.geometry import shape

import base64
import dash
import json
import os
import pprint
import tempfile

import dash_bootstrap_components as dbc
import layouts.sleuth as sl
import numpy as np
import pandas as pd
import plotly.express as px
import rasterio as rio
import rasterio.warp as warp
import sleuth_sklearn.utils as utils


In [2]:
def calculate_coverage(worldcover, sleuth_predictions, start_year):
    world_cover_type = {
        "Tree Cover": 10,
        "Shrubland": 20,
        "Grassland": 30,
        "Cropland": 40,
        "Built-up": 50,
        "Bare/Sparse Vegetation": 60,
        "Snow and Ice": 70,
        "Permanent water bodies": 80,
        "Herbaceous wetlands": 90,
        "Mangroves": 95,
        "Moss and lichen": 100,
    }

    # Obtener el tamaño de sleuth_predictions
    num_samples, height, width = sleuth_predictions.shape

    # Crear un diccionario de kernels
    kernels = {}
    for key, value in world_cover_type.items():
        kernels[key] = np.where(worldcover == value, 1, 0)

    # Inicializar un DataFrame vacío
    result_df = pd.DataFrame()

    # Aplicar el proceso a cada clase en world_cover_type
    for key, kernel in kernels.items():
        sample_results = []
        for i in range(num_samples):
            result = np.sum((1 - sleuth_predictions[i]) * kernel) / (height * width)
            sample_results.append(result)
        result_df[key] = sample_results

    sample_results = []
    for i in range(num_samples):
        result = np.sum(sleuth_predictions[i]) / (height * width)
        sample_results.append(result)
    result_df["Urban"] = sample_results
    result_df["Year"] = list(range(start_year + 1, start_year + num_samples + 1))
    result_df.set_index("Year", inplace=True)
    result_df = result_df * 100
    return result_df

def plot_coverage(lc_df, title):
    # Eliminamos columnas que tengan cero
    lc_df = lc_df.loc[:, (lc_df != 0).any(axis=0)]

    # Ordenamos columnas
    column_names_sorted = lc_df.iloc[0].sort_values(ascending=False).index
    lc_df = lc_df[column_names_sorted]

    # "Urban" se convierte en la primera columna
    wanted_cols = ["Urban"] + [col for col in lc_df.columns if (col != "Urban") and (col != "Year")]
    lc_df = lc_df[wanted_cols]
    lc_df["Año"] = list(range(2021, 2071))

    fig = px.area(lc_df, x="Año", y=wanted_cols, color_discrete_map=WORLD_COVER_COLOR, markers=True)

    fig.update_layout(
        title=title,
        yaxis_title="Porcentaje",
        xaxis_title="Año",
        legend_title="Tipo de cobertura",
        hovermode="x",
    )

    fig.update_traces(hovertemplate="%{y:.2f}<extra></extra>")

    return fig

def load_sleuth_predictions(path_cache, id_hash, mode):
    num_years = 50  # Número de años de predicción
    num_rows, num_cols = 100, 100  # Dimensiones espaciales

    # Crear una matriz tridimensional con valores aleatorios
    # Dimensiones: años, filas, columnas
    return np.random.rand(num_years, num_rows, num_cols)

def plot_sleuth_predictions(grid, start_year, num_years):
    sim_years = list(range(start_year + 1, start_year + num_years + 1))

    grid_plot = xr.DataArray(
        data=grid * 100,
        dims=["Año", "y", "x"],
        coords={
            "Año": sim_years,
            "y": list(range(grid.shape[1])),
            "x": list(range(grid.shape[2])),
        },
    )

    fig = px.imshow(
        grid_plot,
        animation_frame="Año",
        labels=dict(color="Probabilidad de urbanización"),
        zmin=0,
        zmax=100,
        aspect="equal",
    )
    fig.update_xaxes(showticklabels=False, visible=False)
    fig.update_yaxes(showticklabels=False, visible=False)
    return fig

WORLD_COVER_COLOR = {
    "Tree Cover": "#006400",
    "Shrubland": "#ffbb22",
    "Grassland": "#ffff4c",
    "Cropland": "#f096ff",
    "Built-up": "#fa0000",
    "Bare/Sparse Vegetation": "#b4b4b4",
    "Snow and Ice": "#f0f0f0",
    "Permanent water bodies": "#0064c8",
    "Herbaceous wetlands": "#0096a0",
    "Mangroves": "#00cf75",
    "Moss and lichen": "#fae6a0",
}

FIELDS = ["diffusion", "breed", "spread", "slope", "road"]

In [5]:
def summary(id_hash, urban_rasters, years, language = 'es'):
    translations = {
        "es": {
            "year": "Año",
            "urbanization_percentage": "Porcentaje de Urbanización",
            "category": "Categoría",
            "observations": "Observaciones",
            "expansion": "Expansión",
            "general_summary": "Resumen General",
            "urban_area_increase": "+{0:.1f}% de aumento del área urbanizada 2070 vs. 2020.",
            "modes": ["inercial", "acelerada", "controlada"]
        },
        "en": {
            "year": "Year",
            "urbanization_percentage": "Urbanization Percentage",
            "category": "Category",
            "observations": "Observations",
            "expansion": "Expansion",
            "general_summary": "General Summary",
            "urban_area_increase": "+{0:.1f}% increase in urban area 2070 vs. 2020.",
            "modes": ["inertial", "accelerated", "controlled"]
        },
        "pt": {
            "year": "Ano",
            "urbanization_percentage": "Percentagem de Urbanização",
            "category": "Categoria",
            "observations": "Observações",
            "expansion": "Expansão",
            "general_summary": "Resumo Geral",
            "urban_area_increase": "+{0:.1f}% de aumento da área urbana 2070 vs. 2020.",
            "modes": ["inercial", "acelerada", "controlada"]
        }

    }
    
    path_cache = Path(f"./data/cache/{id_hash}")

    #worldcover = np.load(path_cache / "worldcover.npy")
    worldcover = np.random.rand(100, 100)

    start_year = 2020
    num_years = 50

    modes = ["inercial", "acelerada", "controlada"]
    id_hash = str(id_hash)

    historical_years = np.array(years)
    historical_grids = np.array(urban_rasters)

    x = list(historical_years)
    y = [grid.sum() / grid.size for grid in historical_grids]
    z = ["Observaciones"] * len(x)

    final_y = y[-1]

    tabs = []
    coverage_graphs = []
    for mode in modes:
        grids = load_sleuth_predictions(path_cache, id_hash, mode=mode)

        x_pred = list(range(start_year + 1, start_year + num_years + 1))
        y_pred = [grid.sum() / grid.size for grid in grids]
        z_pred = [f"Expansión {mode}"] * (len(x_pred) + 1)

        x_pred = [start_year] + x_pred
        y_pred = [final_y] + y_pred

        x.extend(x_pred)
        y.extend(y_pred)
        z.extend(z_pred)

        # Plot Sleuth Predictions
        tab = dbc.Tab(
            dbc.Card(
                dbc.CardBody(
                    dbc.Container(
                        dbc.Row(
                            dbc.Col(
                                dcc.Graph(
                                    figure=plot_sleuth_predictions(grids, 2020, 50),
                                    responsive=True,
                                    style={"height": "60vh"},
                                ),
                                width=8,
                            )
                        )
                    )
                )
            ),
            label=f"Expansión {mode}",
        )
        tabs.append(tab)
        # Coverage
        estimate_coverage = calculate_coverage(
            worldcover, grids, start_year, 
        )
        fig_coverage = plot_coverage(estimate_coverage, f"Expansión {mode}")
        # fig_coverage.write_image(f"./test/{mode}.eps", width=1200, height=600, scale=1.5)
        coverage_graphs.append(dcc.Graph(figure=fig_coverage))

    df = pd.DataFrame(
        zip(x, y, z), columns=["Año", "Porcentaje de urbanización", "Categoría"]
    )
    fig = px.line(df, x="Año", y="Porcentaje de urbanización", color="Categoría")
    fig.update_yaxes(tickformat=",.0%")
    # fig.write_image("./test/lines.eps", width=1200, height=600, scale=1.5)

    ## Cambio Porcentual por Escenario

    base = df.loc[(df["Año"] == 2020) & (df["Categoría"] == "Observaciones")][
        "Porcentaje de urbanización"
    ].values[0]

    columns = []
    for cat, color in zip(
        ["acelerada", "inercial", "controlada"], ["danger", "warning", "success"]
    ):
        cat = f"Expansión {cat}"
        prediction = df.loc[(df["Año"] == 2070) & (df["Categoría"] == cat)][
            "Porcentaje de urbanización"
        ].values[0]
        col = dbc.Col(dbc.Card(
            dbc.CardBody(
                [
                    html.H5(cat.title(), className="card-title"),
                    html.P(
                        f"+{round(((prediction - base)/ base) * 100, 1)}% de área urbanizada 2070 vs. 2020.",
                        className="card-text",
                    ),
                ]
            ),
            color=color,
            inverse=True,
        ))
        columns.append(col)

    cards = html.Div(dbc.Row(columns, className="mb-4"))
    all_elements = [cards] + coverage_graphs + [dcc.Graph(figure=fig)]
    plot_tab = dbc.Tab(
        dbc.Card(dbc.CardBody(all_elements)), 
        label=translations[language]["general_summary"]
    )
    tabs = [plot_tab] + tabs
    out = dbc.Tabs(tabs, active_tab="tab-0")

    return out

In [3]:
def summary(id_hash, urban_rasters, years, language = "es"):
    
    translations = {
        "es": {
            "year": "Año",
            "urbanization_percentage": "Porcentaje de Urbanización",
            "category": "Categoría",
            "observations": "Observaciones",
            "expansion": "Expansión",
            "general_summary": "Resumen General",
            "urban_area_increase": "+{0:.1f}% de aumento del área urbanizada 2070 vs. 2020.",
            "modes": ["inercial", "acelerada", "controlada"]
        },
        "en": {
            "year": "Year",
            "urbanization_percentage": "Urbanization Percentage",
            "category": "Category",
            "observations": "Observations",
            "expansion": "Expansion",
            "general_summary": "General Summary",
            "urban_area_increase": "+{0:.1f}% increase in urban area 2070 vs. 2020.",
            "modes": ["inertial", "accelerated", "controlled"]
        },
        "pt": {
            "year": "Ano",
            "urbanization_percentage": "Percentagem de Urbanização",
            "category": "Categoria",
            "observations": "Observações",
            "expansion": "Expansão",
            "general_summary": "Resumo Geral",
            "urban_area_increase": "+{0:.1f}% de aumento da área urbana 2070 vs. 2020.",
            "modes": ["inercial", "acelerada", "controlada"]
        }

    }
    
    path_cache = Path(f"./data/cache/{id_hash}")

    worldcover = np.load(path_cache / "worldcover.npy")

    start_year = 2020
    num_years = 50

    modes = ["inercial", "acelerada", "controlada"]
    id_hash = str(id_hash)

    historical_years = np.array(years)
    historical_grids = np.array(urban_rasters)

    x = list(historical_years)
    y = [grid.sum() / grid.size for grid in historical_grids]
    z = ["Observaciones"] * len(x)

    final_y = y[-1]

    tabs = []
    coverage_graphs = []
    for mode in modes:
        grids = load_sleuth_predictions(path_cache, id_hash, mode=mode)

        x_pred = list(range(start_year + 1, start_year + num_years + 1))
        y_pred = [grid.sum() / grid.size for grid in grids]
        z_pred = [f"Expansión {mode}"] * (len(x_pred) + 1)

        x_pred = [start_year] + x_pred
        y_pred = [final_y] + y_pred

        x.extend(x_pred)
        y.extend(y_pred)
        z.extend(z_pred)

        # Plot Sleuth Predictions
        tab = dbc.Tab(
            dbc.Card(
                dbc.CardBody(
                    dbc.Container(
                        dbc.Row(
                            dbc.Col(
                                dcc.Graph(
                                    figure=plot_sleuth_predictions(grids, 2020, 50),
                                    responsive=True,
                                    style={"height": "60vh"},
                                ),
                                width=8,
                            )
                        )
                    )
                )
            ),
            label=f"Expansión {mode}",
        )
        tabs.append(tab)
        # Coverage
        estimate_coverage = calculate_coverage(
            worldcover, grids, start_year, 
        )
        fig_coverage = plot_coverage(estimate_coverage, f"Expansión {mode}")
        # fig_coverage.write_image(f"./test/{mode}.eps", width=1200, height=600, scale=1.5)
        coverage_graphs.append(dcc.Graph(figure=fig_coverage))

    df = pd.DataFrame(
        zip(x, y, z), columns=["Año", "Porcentaje de urbanización", "Categoría"]
    )
    fig = px.line(df, x="Año", y="Porcentaje de urbanización", color="Categoría")
    fig.update_yaxes(tickformat=",.0%")
    # fig.write_image("./test/lines.eps", width=1200, height=600, scale=1.5)

    ## Cambio Porcentual por Escenario

    base = df.loc[(df["Año"] == 2020) & (df["Categoría"] == "Observaciones")][
        "Porcentaje de urbanización"
    ].values[0]

    columns = []
    for cat, color in zip(
        ["acelerada", "inercial", "controlada"], ["danger", "warning", "success"]
    ):
        cat = f"Expansión {cat}"
        prediction = df.loc[(df["Año"] == 2070) & (df["Categoría"] == cat)][
            "Porcentaje de urbanización"
        ].values[0]
        col = dbc.Col(dbc.Card(
            dbc.CardBody(
                [
                    html.H5(cat.title(), className="card-title"),
                    html.P(
                        f"+{round(((prediction - base)/ base) * 100, 1)}% de área urbanizada 2070 vs. 2020.",
                        className="card-text",
                    ),
                ]
            ),
            color=color,
            inverse=True,
        ))
        columns.append(col)

    cards = html.Div(dbc.Row(columns, className="mb-4"))
    all_elements = [cards] + coverage_graphs + [dcc.Graph(figure=fig)]
    plot_tab = dbc.Tab(
        dbc.Card(dbc.CardBody(all_elements)), 
        label=translations[language]["general_summary"]
    )

    tabs = [plot_tab] + tabs
    out = dbc.Tabs(tabs, active_tab="tab-0")

    return out

In [12]:
import numpy as np
from pathlib import Path
from dash import html, dcc, callback, Input, Output, State

# Datos de ejemplo para id_hash, urban_rasters y years
id_hash = "example_id"
num_years = 5
num_rows, num_cols = 100, 100
urban_rasters = np.random.rand(num_years, num_rows, num_cols)
years = [2016, 2017, 2018, 2019, 2020]

# Llamada a la función summary con los datos de ejemplo
resultado_summary = summary(id_hash, urban_rasters, years, language = 'es')

import dash
import dash_html_components as html

# Translation

language_buttons = dbc.ButtonGroup(
    [
        dbc.Button("Español", id="btn-lang-es", n_clicks=0),
        dbc.Button("English", id="btn-lang-en", n_clicks=0),
        dbc.Button("Portuguese", id="btn-lang-pt", n_clicks=0),
    ],
    style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1"},
)

app = dash.Dash(__name__)
app.layout = html.Div([
    language_buttons,
    resultado_summary
])

@app.callback(
    Output("graph-container", "children"),
    [Input("btn-lang-es", "n_clicks"),
     Input("btn-lang-en", "n_clicks"),
     Input("btn-lang-pt", "n_clicks")]
)
def update_graphs(btn_lang_es, btn_lang_en, btn_lang_pt):
    ctx = dash.callback_context
    if not ctx.triggered:
        language = 'es'  # Idioma predeterminado
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        language = 'es' if button_id == 'btn-lang-es' else 'en' if button_id == 'btn-lang-en' else 'pt'

    # Datos de ejemplo
    id_hash = "example_id"
    urban_rasters = np.random.rand(5, 100, 100)
    years = [2016, 2017, 2018, 2019, 2020]

    # Llamada a la función summary con el idioma seleccionado
    resultado_summary = summary(id_hash, urban_rasters, years, language = language)

    return resultado_summary

if __name__ == '__main__':
    app.run_server(debug=True)



In [9]:
import numpy as np
import dash
from dash import html, dcc, callback, Input, Output
import dash_bootstrap_components as dbc

# Datos de ejemplo
id_hash = "example_id"
urban_rasters = np.random.rand(5, 100, 100)
years = [2016, 2017, 2018, 2019, 2020]

# Botones de idioma
language_buttons = dbc.ButtonGroup(
    [
        dbc.Button("Español", id="btn-lang-es", n_clicks=0),
        dbc.Button("English", id="btn-lang-en", n_clicks=0),
        dbc.Button("Portuguese", id="btn-lang-pt", n_clicks=0),
    ],
    style={"position": "absolute", "top": "10px", "right": "10px", "z-index": "1"},
)

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Estructura básica de la aplicación
app.layout = html.Div([
    language_buttons,
    html.Div(id='content')  # Contenedor principal para contenido dinámico
])

# Función de callback para actualizar los contenidos
@app.callback(
    Output("content", "children"),
    [Input("btn-lang-es", "n_clicks"),
     Input("btn-lang-en", "n_clicks"),
     Input("btn-lang-pt", "n_clicks")]
)
def update_content(btn_lang_es, btn_lang_en, btn_lang_pt):
    ctx = dash.callback_context
    if not ctx.triggered:
        language = 'es'  # Idioma predeterminado
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        language = 'es' if button_id == 'btn-lang-es' else 'en' if button_id == 'btn-lang-en' else 'pt'

    # Llamada a la función summary con el idioma seleccionado
    summary_result = summary(id_hash, urban_rasters, years, language=language)

    # Crear componentes Dash basados en los resultados de summary
    components = [
        html.H1("Traduccion"),
        html.P(f" "),
        html.Div(summary_result)  # Asumiendo que summary_result es adecuado para incluir directamente
    ]

    return components

if __name__ == '__main__':
    app.run_server(debug=True)
